# Setup and Imports
Prepare helpers to fetch the Angular error page and parse it.

In [1]:
import requests
from bs4 import BeautifulSoup

URL = "https://angular.dev/errors/NG0401"

# Fetch Error Page Content
Retrieve the page with basic error handling so we can parse it.

In [2]:
response = requests.get(URL, timeout=15)
response.raise_for_status()
html_text = response.text
print(f"Fetched {len(html_text)} chars from {URL}")

Fetched 120728 chars from https://angular.dev/errors/NG0401


# Extract Main Text from HTML
Parse the HTML, drop non-content tags, and gather readable paragraphs.

In [3]:
soup = BeautifulSoup(html_text, "html.parser")

for tag in soup(["script", "style", "noscript"]):
    tag.decompose()

main = soup.find("article") or soup.find("main") or soup.find("body")
text_chunks = []
if main:
    for p in main.find_all(["p", "li", "h1", "h2", "h3", "h4"]):
        chunk = p.get_text(" ", strip=True)
        if chunk:
            text_chunks.append(chunk)
plain_text = "\n".join(text_chunks)
print(f"Collected {len(plain_text.splitlines())} text lines")

Collected 6 text lines


# Heuristic Summary of Cause and Fix
Use simple keyword search to draft a short cause-and-fix summary.

In [4]:
import re

def find_sentences(text, keywords):
    sentences = re.split(r"(?<=[.!?])\s+", text)
    hits = []
    for s in sentences:
        lower = s.lower()
        if any(k in lower for k in keywords):
            hits.append(s.strip())
    return hits

cause_hits = find_sentences(plain_text, ["cause", "happens", "when"])
fix_hits = find_sentences(plain_text, ["fix", "solution", "import", "add", "include"])

summary_parts = []
if cause_hits:
    summary_parts.append(cause_hits[0])
if fix_hits:
    summary_parts.append(fix_hits[0])
short_summary = " ".join(summary_parts) if summary_parts else "No summary extracted."

key_quotes = (cause_hits[:2] + fix_hits[:2])[:4]

# Display Short Summary and Key Quotes
Show the condensed summary and the most relevant sentences from the page.

In [5]:
print("Summary:\n", short_summary, "\n")
if key_quotes:
    print("Key quotes:")
    for q in key_quotes:
        print("-", q)
else:
    print("No key quotes found")

Summary:
 This error is thrown when bootstrapApplication is used on the server without providing a BootstrapContext . 

Key quotes:
- This error is thrown when bootstrapApplication is used on the server without providing a BootstrapContext .
- When using server-side rendering, bootstrapApplication requires a BootstrapContext to be passed as an argument.
